In [1]:
%%capture
# simple neural network implementation of qlearning
import pickle, random

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt

from tensorflow.keras import layers, optimizers
from IPython.display import HTML, display
from statistics import mean

In [12]:
data = pd.read_csv('fra.txt', sep='\t', header=None, names=['English', 'French'])
train_ds = data.sample(frac=.60)
test_ds = data.drop(train_ds.index)

In [17]:
VOCAB_SIZE = 5000
SEQ_LEN = 20
LEARNING_RATE = .001

vectorizers = {
    'en': layers.TextVectorization(max_tokens=VOCAB_SIZE, output_sequence_length=SEQ_LEN),
    'fr': layers.TextVectorization(max_tokens=VOCAB_SIZE, output_sequence_length=SEQ_LEN),
}

vectorizers['en'].adapt(list(train_ds['English']))
vectorizers['fr'].adapt(list(train_ds['French']))

2022-06-30 14:23:30.630077: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-30 14:23:33.574448: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [26]:
vocabs = dict(map(lambda item: (item[0], item[1].get_vocabulary()), vectorizers.items()))

def create_model():
    text = tf.keras.Input(shape=(SEQ_LEN), dtype=tf.float32)
    en_embedding = layers.Embedding(len(vocabs['en']), 64)(text)
    fr_embedding = layers.Embedding(len(vocabs['fr']), 64)(text)
    
    self_attention = layers.Attention()([en_embedding, fr_embedding])
    flattened = layers.Flatten()(self_attention)
    output = layers.Dense(SEQ_LEN)(flattened)
    
    model = tf.keras.Model(inputs=[text], outputs=[output])
    
    adam = optimizers.Adam(learning_rate=LEARNING_RATE)
    loss = 'sparse_categorical_crossentropy'
    metrics = 'accuracy'
    
    model.compile(optimizer=adam, loss=loss, metrics=metrics)
    return model

model = create_model()
model.summary()

5000
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 20, 64)       320000      ['input_3[0][0]']                
                                                                                                  
 attention_2 (Attention)        (None, 20, 64)       0           ['embedding_2[0][0]',            
                                                                  'embedding_2[0][0]']            
                                                                                                  
 flatten_2 (Flatten)            (None, 1280)         0           ['attention_2[0][0]'] 

In [25]:
len(vocabs['en'])

5000